In [42]:
import torch
import torch.nn as nn
# import pandas as pd
import json
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import random
import re
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.optim as optim

#from nltk.tokenize import word_tokenize
from collections import Counter

# from ...src.utils.wiki_tokens.wiki_cleaner import getMorphemeList
# import ../src/utiils/wiki_token/wiki_cleaner

In [6]:
with open("../tokens/wiki.json", "r") as file:
    tokenizer =json.load(file)

In [7]:
with open("text", "r") as file:
    wiki = file.read()

In [ ]:

# def getMorphemeList(text):
#   text = text.lower()
#   text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
#   morpheme_list = word_tokenize(text)
  
#   # Count occurrences of each word
#   word_counts = Counter(morpheme_list)
  
#   # Filter out words that only appear once
#   morpheme_list = [word for word in morpheme_list if word_counts[word] > 1]
  
#   return morpheme_list


# words = getMorphemeList(wiki)

: 

In [8]:
words = wiki.split(sep=" ")
window = 2
print (words[0])
words.pop(0)

def get_words(words,idx,window=2):
    sent = words[max(0,idx-window):min(idx+window+1,len(words))]
    if len(sent) > 1:
        rand_idx = random.randint(0,len(sent)-1)
        target = sent[rand_idx]
        del sent[rand_idx]

        return sent, target
        

In [ ]:
class embed_train_dataset(Dataset):
    def __init__(self, words, tokenizer):
        self.data = words
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        loop = True
        
        while loop == True:
            running = []   
            sent = words[max(0,idx-window):min(idx+window+1,len(words))]
            for word in sent:
                running.append(int(word in tokenizer))
            if np.mean(running) == 1:
                loop = False
            else:
                idx += 1

                    
                    
        if len(sent) > 1:
            rand_idx = random.randint(0,len(sent)-1)
            target = sent[rand_idx]
            del sent[rand_idx]
            tokenized = []
            for item in sent:
                if item in tokenizer:
                    tokenized.append(tokenizer[item])
            if target not in tokenizer:
                exit
            #print (sent, target)
            tokenized = torch.tensor(tokenized)

            
            return tokenized, torch.tensor(tokenizer[target])
        

In [69]:
dataset = embed_train_dataset(words,tokenizer)
dataloader = DataLoader(dataset, batch_size=1,shuffle=True)

for data in dataloader:
    print (data)
    break



[tensor([[    0, 32997, 17988,  1814]]), tensor([21916])]


In [58]:
class CBOW(torch.nn.Module):
    def __init__(self):
        super(CBOW, self).__init__()
        self.embeddings = torch.nn.Embedding(num_embeddings = len(tokenizer), embedding_dim = 200)
        self.linear = torch.nn.Linear(in_features = 200, out_features = len(tokenizer))
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = embeds.mean(dim=1)
        out = self.linear(embeds)
        probs =  F.log_softmax(out, dim=1)
        return probs

In [72]:
def train_loop():
    number_epochs = 5

    #train_wiki, val_wiki = train_test_split(words)


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    dataset = embed_train_dataset(words,tokenizer)
    dataloader = DataLoader(dataset, batch_size=128,shuffle=True)
    

    model = CBOW().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    for epoch in range(number_epochs):
        model.train()
        epoch_loss = 0.0
        for X,Y in tqdm(dataloader):
            X = X.to(device)
            Y = Y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = F.cross_entropy(pred,Y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
        print("Epoch: {epoch}/{number_epochs}, loss: {epoch_loss} ")



In [ ]:
train_loop()

 12%|█▏        | 12122/98261 [02:48<21:33, 66.61it/s]/home/ari/group_work/predict-hacker-news-popularity/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ari/group_work/predict-hacker-news-popularity/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
 12%|█▏        | 12122/98261 [02:59<21:33, 66.61it/s]